<a href="https://colab.research.google.com/github/Ganeshv2002/My_DataScience_Projs/blob/main/Tamil%26English_Hotel_Room_Booking_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import Packages:

In [ ]:
!pip install indic-nlp-library

In [ ]:
# Import packages
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import tensorflow as tf
import nltk
nltk.download('punkt')
from nltk.stem.porter import PorterStemmer

from datetime import date
from datetime import datetime
import random
import string
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Check for GPU availability:

In [ ]:
# Get the GPU device name
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU not available')

SystemError: ignored

In [ ]:
# If there's a GPU available...
if torch.cuda.is_available():

    # Tell PyTorch to use the GPU
    device = torch.device("cuda")

    print('There are {} GPU(s) available.'.format(torch.cuda.device_count()))

    print('GPU name:', torch.cuda.get_device_name(0))

# If not, use the CPU
else:
    print('GPU not found, using the CPU instead.')
    device = torch.device("cpu")

GPU not found, using the CPU instead.


Training Data:

In [ ]:
# Create a dictionary with your training data
intents = {
    'intents': [
        {
            'tag': 'greeting',
            'patterns': [
                'Hi',
                'Hey',
                'Hello',
                'How are you',
                'Is anyone there?',
                'Good day',

            ],
            'responses': [
                'Hey',
                'Hello, thanks for visiting',
                'Hi there, what can I do for you?',
                'Hi there, how can I help?',

            ]
        },
        {
            'tag': 't_greeting',
            'patterns': [
                'வணக்கம்',
                'எப்படி இருக்கிறீர்கள்',
                'யாராவது இருக்கிறார்களா?'
                'யாராவது இருக்கிறீர்களா?'

            ],
            'responses': [
                'வணக்கம்',
                'வணக்கம், நான் உங்களுக்கு எப்படி உதவ முடியும்',
                'வருகைக்கு நன்றி,நான் உங்களுக்கு எப்படி உதவ முடியும்'


            ]
        },
        {
            'tag': 'goodbye',
            'patterns': [
                'Bye',
                'See you later',
                'Goodbye'
            ],
            'responses': [
                'See you later, thanks for visiting',
                'Have a nice day',
                'Bye! Come back again soon',

            ]
        },
        {
            'tag': 'goodbye_t',
            'patterns': [
                'வருகிறேன்',
                'பிறகு பார்க்கலாம்',
                'பிரியாவிடை',
                'விடைப்பெருகிரேன்'
            ],
            'responses': [
                'பிறகு சந்திப்போம், வருகைக்கு நன்றி',
                'வருகிறேன்! விரைவில் சந்திப்போம்'

            ]
        },

        {
            'tag': 'thanks',
            'patterns': [
                'Thanks',
                'Thank you',
                "That's helpful",
                'Thanks a lot!'
            ],
            'responses': [
                'Happy to help!',
                'Anytime!',
                "My pleasure"
            ]
        },
         {
            'tag': 't_thanks',
            'patterns': [
                'நன்றி',
                "உதவியாக இருக்கிறது",
                'மிக்க நன்றி!'
            ],
            'responses': [
                'உதவுவதில் மகிழ்ச்சி!',
                'எப்போது வேண்டுமானாலும், உதவுவதில் மகிழ்ச்சி!',
            ]
        },
        {
            'tag': 'booking',
            'patterns': [
                'I would like to make a reservation',
                'Can I book a room?',
                "I want to make a booking",
                "I'm interested in reserving a room at your hotel",
                "Can you help me make a reservation for a room?",
                "I'd like to book a room for my upcoming stay",
               "Is it possible to make a booking for a room?",
                "Could you assist me with booking a room at your hotel?",
              "I'm looking to reserve a room, can you help me with that?",
               "What is the process to make a reservation for a room?"
            ],
            'responses': [
                'Sure, I only need to know a few details',
                "Lovely. Let's begin with the reservation"
            ]
        },
        {
            'tag': 'booking_t',
            'patterns': [
                'நான் முன்பதிவு செய்ய விரும்புகிறேன்',
                'நான் ஒரு அறையை முன்பதிவு செய்யலாமா?',
                "நான் முன்பதிவு செய்ய விரும்புகிறேன்",
                "நான் உங்கள் ஹோட்டலில் ஒரு அறையை ஒதுக்க ஆர்வமாக இருக்கிறேன்",
                "ஒரு அறைக்கு முன்பதிவு செய்ய எனக்கு உதவ முடியுமா?",
                "நான் வரவிருக்கும் தங்குவதற்கு ஒரு அறையை முன்பதிவு செய்ய விரும்புகிறேன்",
                "நா ஒரு ரூம் புக் பண்ணனும்னு நினைக்குறேன்",
                "நான் ஒரு ரூம் புக் பண்ணனும்னு நினைக்குறேன்"
            ],
            'responses': [
                'நிச்சயமாக, எனக்கு சில விவரங்கள் மட்டுமே தெரிய வேண்டும்',
                 "நல்ல விஷயம். முன்பதிவில் தொடங்குவோம்"
            ]

        }
        {
            'tag': 'cancellation',
            'patterns': [
                'I would like to cancel my booking',
                'Can I cancel a room?',
                "I want to make a cancellation",
                'Cancel a booking',
                'Is it possible for me to cancel my reservation?',
                 'Could you assist me in canceling my booking?',
                 'I would like to request a cancellation of my room reservation',
                 'Can you help me cancel my room booking, please?',
                 'I want to cancel my room reservation, is that possible?',
                 'How can I go about canceling my booking?',
                'Is there a way for me to cancel my reservation for the room?',
                 'I need to cancel my booking, can you guide me through the process?',
                 'Can you cancel my room reservation for me?',
                  'I regret to inform you that I need to cancel my room booking',
            ],
            'responses': [
                'Sure, which is your reference number?',
                "I only need to know the reference number of your booking"
            ]
        },
        {
            'tag': 'cancellation_t',
            'patterns': [
                'எனது முன்பதிவை ரத்து செய்ய விரும்புகிறேன்',
                 'நான் ஒரு அறையை ரத்து செய்யலாமா?',
                 "நான் ரத்து செய்ய விரும்புகிறேன்",
                 'புக்கிங்கை ரத்து செய்',
                 'எனது முன்பதிவை ரத்து செய்ய முடியுமா?',
                  'எனது முன்பதிவை ரத்து செய்ய நீங்கள் எனக்கு உதவ முடியுமா?',
                  'எனது அறை முன்பதிவை ரத்து செய்யக் கோர விரும்புகிறேன்',
                  'எனது அறை முன்பதிவை ரத்துசெய்ய எனக்கு உதவ முடியுமா?',
                  'எனது அறை முன்பதிவை ரத்து செய்ய விரும்புகிறேன், அது சாத்தியமா?',
                  'எனது முன்பதிவை ரத்து செய்வது எப்படி?',
                 'அறைக்கான எனது முன்பதிவை ரத்து செய்ய ஏதாவது வழி இருக்கிறதா?',
                  'எனது முன்பதிவை நான் ரத்து செய்ய வேண்டும், செயல்முறையின் மூலம் எனக்கு வழிகாட்ட முடியுமா?',
                  'எனக்கான அறை முன்பதிவை ரத்து செய்ய முடியுமா?',
                  'எனது அறை முன்பதிவை நான் ரத்து செய்ய வேண்டும் என்பதை உங்களுக்குத் தெரிவிக்க வருந்துகிறேன்',
            ],
            'responses': [
                'நிச்சயமாக, உங்கள் ஆதார் எண் எது?',
                "உங்கள் முன்பதிவு எண் மட்டும் எனக்குத் தெரிய வேண்டும்"
            ]
        },
        {
            'tag': 'payments',
            'patterns': [
                'When do I have to pay?',
                'Can I pay by card?',
                "Do you accept cash?",
                'When is the payment?',
                'How can i make my payment?',
                'How should i pay?',
                'Where should i pay?',
                'Is there a mobile payment option available?',
                'How do I make a payment?',
                'What is the process for payment?',

            ],
            'responses': [
                'The payment is on arrival. We accept cash and card',
                'You can pay on arrival by cash or card',
                'Payment can be made upon arrival, and we accept both cash and card transactions',
                'You can settle the payment upon arrival using cash or card as we accept both payment methods',
                'Cash and card payments are both accepted, and payment is due upon your arrival',
                'We kindly request that payment be made upon arrival, and we accept both cash and card transactions for your convenience',
                'Please note that payment can be made using cash or card upon your arrival, as we accept both payment options',

            ]
        },
        {
            'tag': 'payments_t',
            'patterns': [
                'நான் எப்போது பணம் செலுத்த வேண்டும்?',
                 'கார்டு மூலம் பணம் செலுத்த முடியுமா?',
                 "நீங்கள் பணத்தை ஏற்றுக்கொள்கிறீர்களா?",
                 'கட்டணம் எப்போது?',
                 'எனது கட்டணத்தைச் செலுத்துவது எப்படி?',
                 'எப்படி செலுத்த வேண்டும்?',
                 'எங்கே செலுத்த வேண்டும்?',
                 'மொபைல் கட்டணம் செலுத்தும் விருப்பம் உள்ளதா?',
                 'நான் எப்படி பணம் செலுத்துவது?',
                 'பணம் செலுத்துவதற்கான செயல்முறை என்ன?',

            ],
            'responses': [
               'பணம் வந்தவுடன் உள்ளது. நாங்கள் பணம் மற்றும் அட்டையை ஏற்றுக்கொள்கிறோம்.',
                 'நீங்கள் வந்தவுடன் பணம் அல்லது அட்டை மூலம் பணம் செலுத்தலாம்',
                 'வந்தவுடன் பணம் செலுத்தலாம், நாங்கள் பணம் மற்றும் அட்டை பரிவர்த்தனைகளை ஏற்றுக்கொள்கிறோம்',
                 'இரண்டு கட்டண முறைகளையும் நாங்கள் ஏற்றுக்கொள்வதால், பணம் அல்லது அட்டையைப் பயன்படுத்தி நீங்கள் வந்தவுடன் கட்டணத்தைச் செலுத்தலாம்',
                 'பணம் மற்றும் கார்டு கொடுப்பனவுகள் இரண்டும் ஏற்றுக்கொள்ளப்படும், நீங்கள் வந்தவுடன் பணம் செலுத்தப்படும்',
                 'வந்தவுடன் பணம் செலுத்துமாறு நாங்கள் அன்புடன் கேட்டுக்கொள்கிறோம், மேலும் உங்கள் வசதிக்காக பணம் மற்றும் அட்டை பரிவர்த்தனைகளை நாங்கள் ஏற்றுக்கொள்கிறோம்',
                 'நீங்கள் வந்தவுடன் பணம் அல்லது அட்டையைப் பயன்படுத்தி பணம் செலுத்தலாம் என்பதை நினைவில் கொள்ளவும், நாங்கள் இரண்டு கட்டண விருப்பங்களையும் ஏற்றுக்கொள்கிறோம்',
            ]
        },
        {
            'tag': 'D_payments',
            'patterns': [
                'Do you accept credit cards for payment?',
                'Do you also accept debit cards for payment?',
                'Do you also accept debit cards as payment?',




            ],
            'responses': [
                'Yes, we accept both credit and debit cards for payment',
                'Yes, you can pay with either a credit or debit card',
                'We welcome both credit and debit cards as payment methods'

            ]
        },
         {
            'tag': 'D_payments_t',
            'patterns': [
                'பணம் செலுத்துவதற்கு கடன் அட்டைகளை ஏற்றுக்கொள்கிறீர்களா?',
                 'பணம் செலுத்துவதற்கு டெபிட் கார்டுகளையும் ஏற்றுக்கொள்கிறீர்களா?',
                 'டெபிட் கார்டுகளையும் கட்டணமாக ஏற்றுக்கொள்கிறீர்களா?',



            ],
            'responses': [
                'ஆம், பணம் செலுத்துவதற்கு நாங்கள் கடன் மற்றும் டெபிட் கார்டுகளை ஏற்றுக்கொள்கிறோம்',
                 'ஆம், நீங்கள் கிரெடிட் அல்லது டெபிட் கார்டு மூலம் பணம் செலுத்தலாம்',
                 'கிரெடிட் மற்றும் டெபிட் கார்டுகளை கட்டண முறைகளாக நாங்கள் வரவேற்கிறோம்'
            ]
        },
        {
            'tag': 'location',
            'patterns': [
                'Where is the hotel located?',
                'Can you give me the address?',
                "I want to know the exact location",
                "Can you tell me the location of the hotel?",
                "I'm looking for the hotel, where is it situated?",
                "Where can I find the hotel?",
                'Could you give me directions to the hotel?',
                'What is the address of the hotel?',
                "I'm trying to locate the hotel, could you help me with its whereabouts?",
                "Where exactly is the hotel situated?",
                "Could you tell me the precise location of the hotel?",

            ],
            'responses': [
                'The Green Grande Inn Hotel is located at 21/113 Periyar Pathai, 100 Feet Rd, near Thirunagar, Choolaimedu, Chennai, Tamil Nadu 600094',
                'You will find us at Signal Junction, 21/113 Periyar Pathai, 100 Feet Rd, near Thirunagar, Choolaimedu, Chennai, Tamil Nadu 600094',
                "If you're looking for The Green Grande Inn Hotel, it can be found at 21/113 Periyar Pathai, 100 Feet Rd, near Thirunagar, Choolaimedu, Chennai, Tamil Nadu 600094.",
                "The Green Grande Inn Hotel is situated at 21/113 Periyar Pathai, 100 Feet Rd, near Thirunagar, Choolaimedu, Chennai, Tamil Nadu 600094.",
                "At 21/113 Periyar Pathai, 100 Feet Rd, near Thirunagar, Choolaimedu, Chennai, Tamil Nadu 600094, you'll find The Green Grande Inn Hotel.",
                "Nestled in the spellbinding city, host to diverse cultures, Chennai. Green Grande Inn is a home away from home whether it is accommodation for leisure trip or corporate stay for employees. Arumbakkam Metro Railway Station is just 350 m from the hotel. The property can also be reached through Egmore Railway Station located at a distance of 8 km and Chennai International Airport is 13 km away"
            ]
        },
        {
            'tag': 'location_t',
            'patterns': [
                'ஹோட்டல் எங்கே அமைந்துள்ளது?',
                 'விலாசம் தர முடியுமா?',
                 "எனக்கு சரியான இடம் தெரிய வேண்டும்",
                 "ஹோட்டல் இருக்கும் இடத்தைச் சொல்ல முடியுமா?",
                 "நான் ஹோட்டலைத் தேடுகிறேன், அது எங்கே அமைந்துள்ளது?",
                 "ஹோட்டல் எங்கே கிடைக்கும்?",
                 'ஹோட்டலுக்கு செல்லும் வழியை எனக்கு வழங்க முடியுமா?',
                 'ஹோட்டலின் முகவரி என்ன?',
                 "நான் ஹோட்டலைக் கண்டுபிடிக்க முயற்சிக்கிறேன், அது இருக்கும் இடத்தைப் பற்றி எனக்கு உதவ முடியுமா?",
                 "சரியாக ஹோட்டல் எங்கே உள்ளது?",
                 "ஹோட்டல் இருக்கும் இடத்தைத் துல்லியமாகக் கூற முடியுமா?",

            ],
            'responses': [
                 'கிரீன் கிராண்டே இன் ஹோட்டல், 21/113 பெரியார் பாதை, 100 அடி தூரத்தில், திருநகர், சூளைமேடு, சென்னை, தமிழ்நாடு 600094 என்ற முகவரியில் அமைந்துள்ளது.
                 'நீங்கள் எங்களை சிக்னல் சந்திப்பு, 21/113 பெரியார் பாதை, 100 அடி சாலை, திருநகர், சூளைமேடு, சென்னை, தமிழ்நாடு 600094 அருகில் காணலாம்',
                 "தி கிரீன் கிராண்டே இன் ஹோட்டலை நீங்கள் தேடுகிறீர்களானால், அது 21/113 பெரியார் பாதை, 100 அடி சாலை, திருநகர், சூளைமேடு, சென்னை, தமிழ்நாடு 600094க்கு அருகில் உள்ளது.",
                 "கிரீன் கிராண்டே இன் ஹோட்டல் 21/113 பெரியார் பாதை, 100 அடி தூரத்தில், திருநகர், சூளைமேடு, சென்னை, தமிழ்நாடு 600094 அருகில் அமைந்துள்ளது.",
                 "21/113 பெரியார் பாதை, 100 அடி தூரம், திருநகர், சூளைமேடு, சென்னை, தமிழ்நாடு 600094 அருகில், நீங்கள் The Green Grande Inn ஹோட்டலைக் காணலாம்.",
                 "பல்வேறு கலாச்சாரங்களுக்கு இடமளிக்கும் சென்னை. கிரீன் கிராண்டே விடுதியில் உள்ளது. கிரீன் கிராண்டே விடுதியானது, ஓய்வுப் பயணமாக இருந்தாலும் சரி, பணியாளர்கள் தங்குவதற்கான இடமாக இருந்தாலும் சரி. அரும்பாக்கம் மெட்ரோ ரயில் நிலையம் ஹோட்டலில் இருந்து 350 மீ. தொலைவில் உள்ளது. சொத்து உள்ளது. 8 கிமீ தொலைவில் உள்ள எழும்பூர் ரயில் நிலையம் வழியாகவும் சென்னை சர்வதேச விமான நிலையம் 13 கிமீ தொலைவில் உள்ளது"
             ]
            ]
        },
        {
            'tag': 'contacts_t',
            'patterns': [
                'நான் சந்தைப்படுத்தல் துறையை தொடர்பு கொள்ள விரும்புகிறேன்',
                 'விற்பனையாளர் ஒருவருடன் நான் பேச விரும்புகிறேன்',
                 "எனக்கு நிதி மின்னஞ்சல் கிடைக்குமா?",
                 'மார்கெட்டிங் துறையுடன் நான் தொடர்பு கொள்ள ஏதாவது வழி இருக்கிறதா?',
                 'விற்பனையில் இருக்கும் ஒருவரிடம் பேச விரும்புகிறேன், அது சாத்தியமா?',
                 'நிதித் துறைக்கான தொடர்பு விவரங்களை எனக்குத் தர முடியுமா?',
                 'நான் மார்க்கெட்டிங் குழுவைத் தொடர்பு கொள்ள முயற்சிக்கிறேன், அதற்கு நீங்கள் எனக்கு உதவ முடியுமா?',
                 'விற்பனையாளர் ஒருவரை நான் தொடர்பு கொள்ள வேண்டும், அதற்கு நீங்கள் எனக்கு உதவ முடியுமா?',
                 'என்னை நிதித் துறையுடன் இணைக்க முடியுமா அல்லது அவர்களின் மின்னஞ்சல் முகவரியைத் தர முடியுமா?',
                 'நீங்கள் என்னை மார்க்கெட்டிங் துறைக்கு அனுப்பினால் நான் பாராட்டுவேன்.',
                 'எனது விசாரணை தொடர்பாக விற்பனையில் நான் யாரை தொடர்பு கொள்ள வேண்டும்?',
                 'நிதித்துறையைச் சேர்ந்த ஒருவரைத் தொடர்பு கொள்ள விரும்புகிறேன், அவர்களின் மின்னஞ்சலை எனக்கு வழங்க முடியுமா?',
                 'மார்கெட்டிங் குழுவை நான் எவ்வாறு தொடர்பு கொள்வது?',
                 'எப்படி தொடர்பு கொள்வது?'
            ],
            'responses': [
                'நிச்சயம். அவர்களைத் தொடர்புகொள்ள, மின்னஞ்சல் அனுப்பவும்:',
                'நிச்சயமாக. துறையுடன் தொடர்பு கொள்ள வேண்டிய மின்னஞ்சல்:'
            ]
        },
        {
            'tag': 'quit',
            'patterns': [
                'How can i end this chat?',
                'how can i leave this chat?',
                "What is the process to close this chat?",
                "Is there a way to terminate this chat session?",
                 "Can you tell me how to end this conversation?",
                "How do I exit this chat?",
                "I'm ready to finish this chat, how can I do that?"
            ],
            'responses': [
                "type 'quit' to end this session"
            ]
        },
         {
            'tag': 'quit_t',
            'patterns': [
                'இந்த அரட்டையை நான் எப்படி முடிக்க முடியும்?',
                 'இந்த அரட்டையை நான் எப்படி விட்டுவிட முடியும்?',
                 "இந்த அரட்டையை மூடுவதற்கான செயல்முறை என்ன?",
                 "இந்த அரட்டை அமர்வை நிறுத்த வழி உள்ளதா?",
                  "இந்த உரையாடலை எப்படி முடிப்பது என்று சொல்ல முடியுமா?",
                 "இந்த அரட்டையிலிருந்து நான் எப்படி வெளியேறுவது?",
                 "இந்த அரட்டையை முடிக்க நான் தயாராக இருக்கிறேன், அதை எப்படி செய்வது?"
            ],
            'responses': [
                "இந்த அமர்வை முடிக்க 'நிறுத்து' என தட்டச்சு செய்க"
            ]
        },
        {
            'tag': 'cost',
            'patterns': [
                "How much does it cost to book a room?",
                "What are the rates for booking a room at your hotel?",
                "Can you provide me with information on the pricing for your hotel rooms?",
                "I'm interested in booking a room at your hotel. What are the room rates?",
                "What is the cost for reserving a room at your hotel?",
                "Could you tell me the price range for a room booking at your hotel?",
                "I would like to inquire about the rates for booking a room for the dates I have in mind.",
                "What are the current prices for booking a room at your hotel?",
                 "Can you give me a breakdown of the costs involved in booking a room at your hotel?",
                 "How much do your rooms typically cost to book?",
                 "I'm wondering what the pricing structure is for your hotel rooms."
            ],
            'responses': [
                "For your desired dates, the price for a standard room starts at ₹1000 per night, a deluxe room starts at ₹2000 per night",
                "The price of a room varies depending on the type of room and the dates you wish to stay. For example, a standard room for your desired dates would cost ₹1000 per night, while a deluxe room would cost ₹2000 per night"
            ]
        },
        {
            'tag': 'cost_t',
            'patterns': [
                "ஒரு அறையை முன்பதிவு செய்ய எவ்வளவு செலவாகும்?",
                 "உங்கள் ஹோட்டலில் அறையை முன்பதிவு செய்வதற்கான கட்டணங்கள் என்ன?",
                 "உங்கள் ஹோட்டல் அறைகளுக்கான விலை குறித்த தகவலை எனக்கு வழங்க முடியுமா?",
                 "உங்கள் ஹோட்டலில் அறையை முன்பதிவு செய்ய ஆர்வமாக உள்ளேன். அறையின் விலை என்ன?",
                 "உங்கள் ஹோட்டலில் ஒரு அறையை முன்பதிவு செய்வதற்கான செலவு என்ன?",
                 "உங்கள் ஹோட்டலில் அறை முன்பதிவு செய்வதற்கான விலை வரம்பை என்னிடம் சொல்ல முடியுமா?",
                 "நான் மனதில் வைத்திருக்கும் தேதிகளுக்கான அறையை முன்பதிவு செய்வதற்கான கட்டணங்களைப் பற்றி விசாரிக்க விரும்புகிறேன்.",
                 "உங்கள் ஹோட்டலில் அறையை முன்பதிவு செய்வதற்கான தற்போதைய விலைகள் என்ன?",
                  "உங்கள் ஹோட்டலில் அறையை முன்பதிவு செய்வதில் உள்ள செலவுகளின் விவரத்தை எனக்குத் தர முடியுமா?",
                  "உங்கள் அறைகளை முன்பதிவு செய்ய பொதுவாக எவ்வளவு செலவாகும்?",
                  "உங்கள் ஹோட்டல் அறைகளுக்கான விலை அமைப்பு என்ன என்று நான் யோசித்துக்கொண்டிருக்கிறேன்."
            ],
            'responses': [
                 "நீங்கள் விரும்பும் தேதிகளுக்கு, நிலையான அறையின் விலை ஒரு இரவுக்கு ₹1000 இல் தொடங்குகிறது, டீலக்ஸ் அறையின் விலை ஒரு இரவுக்கு ₹2000 இல் தொடங்குகிறது",
                 "ஒரு அறையின் விலை, நீங்கள் தங்க விரும்பும் அறையின் வகை மற்றும் தேதிகளைப் பொறுத்து மாறுபடும். எடுத்துக்காட்டாக, நீங்கள் விரும்பும் தேதிகளுக்கான நிலையான அறைக்கு ஒரு இரவுக்கு ₹1000 செலவாகும், அதே சமயம் டீலக்ஸ் அறைக்கு ஒரு இரவுக்கு ₹2000 செலவாகும்"
             ]
         }, ]
        },
        {
            'tag': 'Date',
            'patterns': [
                "what is today's date",
                "What's the date today?",
                "Can you tell me what day it is today?",
                "What's the current date?",
                "Could you inform me of today's date?",
                 "What's today's date, please?"
            ],
            'responses': [
                "Today's date is "
            ]
        },
        {
            'tag': 'Date_t',
            'patterns': [
                "இன்றைய தேதி என்ன",
                 "இன்று என்ன தேதி?",
                 "இன்று என்ன நாள் என்று சொல்ல முடியுமா?",
                 "தற்போதைய தேதி என்ன?",
                 "இன்றைய தேதியை எனக்குத் தெரிவிக்க முடியுமா?",
                  "தயவுசெய்து இன்றைய தேதி என்ன?"
            ],
            'responses': [
                "இன்றைய தேதி"
            ]
        },
        {
            'tag': 'types_of_room',
            'patterns': [
                'What types of rooms do you have?',
                'Can you tell me about your different room types?',
                'What is the difference between your standard and deluxe rooms?'
            ],
            'responses': [
                'We have several room types available, including standard, deluxe, and suite. The main differences are usually the size and price between standard and deluxe hotel rooms. Deluxe rooms are usually larger than than standard rooms, may include a bathtub and a shower in the bathroom, and include more high-end amenities. .'
            ]
        },
        {
            'tag': 'types_of_room_t',
            'patterns': [
                'உங்களிடம் என்ன வகையான அறைகள் உள்ளன?',
                 'உங்கள் வெவ்வேறு அறை வகைகளைப் பற்றி என்னிடம் சொல்ல முடியுமா?',
                 'உங்கள் நிலையான அறைகளுக்கும் டீலக்ஸ் அறைகளுக்கும் என்ன வித்தியாசம்?'
            ],
            'responses': [
                'ஸ்டாண்டர்ட், டீலக்ஸ் மற்றும் சூட் உள்ளிட்ட பல அறை வகைகள் எங்களிடம் உள்ளன. முக்கிய வேறுபாடுகள் பொதுவாக நிலையான மற்றும் டீலக்ஸ் ஹோட்டல் அறைகளுக்கு இடையேயான அளவு மற்றும் விலை. டீலக்ஸ் அறைகள் பொதுவாக நிலையான அறைகளை விட பெரியதாக இருக்கும், குளியலறையில் ஒரு குளியல் தொட்டி மற்றும் குளியலறையை உள்ளடக்கியிருக்கலாம், மேலும் உயர்தர வசதிகளை உள்ளடக்கியது. .'
            ]
        },
        {
            'tag': 'availability',
            'patterns': [
                'Are there any rooms available for the nights of  and ?',
                'Can I book a room for  to  ?',
                'I want to stay at the hotel from   to . Is that possible?'
            ],
            'responses': [
                'Let me check...yes, We do have rooms available for those dates. Would you like to book now?',
                'We do have rooms available for those dates. Would you like to book now?'
            ]
        },
        {
            'tag': 'availability_t',
            'patterns': [
                '12/1 மற்றும் 13/1 இரவுகளுக்கு ஏதேனும் அறைகள் உள்ளனவா?',
                 'நான் ஒரு அறையை முன்பதிவு செய்யலாமா?',
                 'நான் முதல் ஹோட்டலில் தங்க விரும்புகிறேன். அது சாத்தியமா?'
            ],
            'responses': [
                'நான் சரிபார்க்கிறேன்... ஆம், அந்த தேதிகளுக்கு எங்களிடம் அறைகள் உள்ளன. இப்போதே முன்பதிவு செய்ய விரும்புகிறீர்களா?',
                'அந்த தேதிகளுக்கு எங்களிடம் அறைகள் உள்ளன. இப்போதே முன்பதிவு செய்ய விரும்புகிறீர்களா?'
            ]
        },
        {
            'tag': 'types_of_services',
            'patterns': [
                'What types of services do you have?',
                'What types of services do you provide?',
                'services?',
                'what are the available services'
            ],
            'responses': [
                'We provide Dining, Room service, Laundry and Money Exchange'
            ]
        },
        {
            'tag': 'types_of_services_t',
            'patterns': [
                'உங்களிடம் என்ன வகையான சேவைகள் உள்ளன?',
                 'நீங்கள் என்ன வகையான சேவைகளை வழங்குகிறீர்கள்?',
                 'சேவைகள்?',
                 'கிடைக்கும் சேவைகள் என்ன'
            ],
            'responses': [
                'நாங்கள் உணவு, அறை சேவை, சலவை மற்றும் பணப் பரிமாற்றம் ஆகியவற்றை வழங்குகிறோம்'
            ]
        },
        {
            'tag':'check_intime',
            'patterns': [
                'At what time should i check in?',
                'what is the check in and check out time?',
                'what is the check in time'

            ],
            'responses': [
                'Check in and Check out time is at 12PM'
            ]

        },
        {
            'tag':'check_intime_t',
            'patterns': [
                'நான் எந்த நேரத்தில் செக்-இன் செய்ய வேண்டும்?',
                 'செக் இன் மற்றும் செக் அவுட் நேரம் என்ன?',
                 'செக் இன் டைம் என்ன'

            ],
            'responses': [
                'செக் இன் மற்றும் செக் அவுட் நேரம் மதியம் 12 மணி'
            ]

        }



    ]
}

Data Pre-Processing:

In [ ]:
stemmer = PorterStemmer()
def tokenize(sentence):
 # Detect the language of the text
    lang = detect(text)

    # Normalize the text using IndicNormalizer for Tamil text
    if lang == 'ta':
        factory = IndicNormalizerFactory()
        normalizer = factory.get_normalizer('ta')
        text = normalizer.normalize(text)

        # Tokenize the text using IndicTokenizer
        return indic_tokenize.trivial_tokenize(text)

    # Normalize and tokenize the text using word_tokenize for English text
    elif lang == 'en':
        text = text.lower()
        text = re.sub(r'\d+', '', text)  # Remove digits
        text = text.translate(str.maketrans('', '', string.punctuation))  # Remove punctuation
        return nltk.word_tokenize(text)

def bag_of_words(tokenized_sentence, all_words):
    tokenized_sentence = [stemmer.stem(w.lower()) for w in tokenized_sentence]
    bag = np.zeros(len(all_words), dtype=np.float32)

    for idx, w in enumerate(all_words):
        if w in tokenized_sentence:
            bag[idx] = 1.0

    return bag

Build Model:

In [ ]:
all_words = []
tags = []
xy = []

# Save all the keywords in different variables
for intent in intents['intents']:
    tag= intent['tag']
    tags.append(tag)
    for pattern in intent['patterns']:
        w = tokenize(pattern)
        all_words.extend(w)
        xy.append((w, tag))

# Create the vocabulary
ignore_words = ['?', '!', '.', ',']
all_words = [stemmer.stem(w.lower()) for w in all_words if w not in ignore_words]
all_words = sorted(set(all_words))
tags = sorted(set(tags))

In [ ]:
# Set the final training data
X_train = []
y_train = []
for (pattern_sentence, tag) in xy:
    bag = bag_of_words(pattern_sentence, all_words)
    X_train.append(bag)
    label = tags.index(tag)
    y_train.append(label)

X_train = np.array(X_train)
y_train = np.array(y_train)

In [ ]:
class ChatDataset(Dataset):
    def __init__(self):
        self.n_samples = len(X_train)
        self.x_data = X_train
        self.y_data = y_train

    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.n_samples

Neural net & Training Dataset:

In [ ]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size)
        self.l2 = nn.Linear(hidden_size, hidden_size)
        self.l3 = nn.Linear(hidden_size, num_classes)
        self.relu = nn.ReLU()

    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        out = self.relu(out)
        out = self.l3(out)
        # no activation and no softmax
        return out

In [ ]:
# Hyperparameters
batch_size=8
hidden_size=64
output_size=len(tags)
input_size = len(X_train[0])
learning_rate = 0.001
num_epochs = 1000

dataset = ChatDataset()
train_loader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True, num_workers=2)

model = NeuralNet(input_size, hidden_size, output_size).to(device)

In [ ]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
for epoch in range(num_epochs):
    total_loss = 0
    total_correct = 0
    total_samples = 0

    for (words, labels) in train_loader:
        words = words.to(device)
        labels = labels.to(device)

        # Forward
        outputs = model(words)
        loss = criterion(outputs, labels)

        # Backward and optimizer
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Calculate accuracy
        predicted = torch.argmax(outputs, dim=1)
        correct = (predicted == labels).sum().item()
        total_correct += correct
        total_samples += labels.size(0)

        # Print batch loss and accuracy
        batch_loss = loss.item()
        batch_acc = correct / labels.size(0)
        total_loss += batch_loss

    # Print epoch loss and accuracy
    epoch_loss = total_loss / len(train_loader)
    epoch_acc = total_correct / total_samples


    print(f"Epoch [{epoch+1}/{num_epochs}], loss: {epoch_loss:.4f}, acc: {epoch_acc:.4f}")


Epoch [100/1000], loss: 0.2320, acc: 0.9762
Epoch [200/1000], loss: 0.0117, acc: 1.0000
Epoch [300/1000], loss: 0.0023, acc: 1.0000


KeyboardInterrupt: ignored

Accuracy

In [ ]:
all_bookings = pd.DataFrame()

class Customer:


    def __init__(self, name, dates, room, service,rate):
        self.name = name
        self.dates = (datetime.strptime(dates[0], '%d/%m/%Y').date(), datetime.strptime(dates[1], '%d/%m/%Y').date())
        self.room = room
        self.service = service
        self.rate = rate

    def ID(self):
        letters = string.ascii_uppercase
        digits = string.digits
        a = random.sample(letters, 3) + random.sample(digits, 4)
        self.id = ''.join(a)

    def nights(self):
        nights = (self.dates[1] - self.dates[0]).days
        return nights

    def final_price(self):
        price = self.rate * float(self.nights())
        return price

    def __str__(self):
        return f'''
        Mr./Miss. {self.name[1]},

        We are delighted to confirm your booking with us for the {self.dates[0]} till the {self.dates[1]}.
        A {self.room} with {self.service} for the final rate of ₹{self.rate} per night.
        Total price: ₹{self.final_price()}
        Your reference number is {self.id}.
        Keep this number in case you want to modify or cancel your booking in the future.

        Best Regards,
        Green Grande Inn
        '''

class tamil_Customer:


    def __init__(self, name, dates, room, service,rate):
        self.name = name
        self.dates = (datetime.strptime(dates[0], '%d/%m/%Y').date(), datetime.strptime(dates[1], '%d/%m/%Y').date())
        self.room = room
        self.service = service
        self.rate = rate

    def ID(self):
        letters = string.ascii_uppercase
        digits = string.digits
        a = random.sample(letters, 3) + random.sample(digits, 4)
        self.id = ''.join(a)

    def nights(self):
        nights = (self.dates[1] - self.dates[0]).days
        return nights

    def final_price(self):
        price = self.rate * float(self.nights())
        return price

    def __str__(self):
        return f'''
        திரு/மிஸ். {self.name[1]},

        {self.dates[1]} வரை {self.dates[0]} எங்களிடம் உங்கள் முன்பதிவை உறுதிப்படுத்துவதில் நாங்கள் மகிழ்ச்சியடைகிறோம்.
        {self.service} உடன் ஒரு {self.room} ஒரு இரவுக்கான இறுதிக் கட்டணமான ₹{self.rate}.
        மொத்த விலை: ₹{self.final_price()}
        உங்கள் ஆதார் எண் {self.id}.
        எதிர்காலத்தில் உங்கள் முன்பதிவை மாற்றவோ அல்லது ரத்து செய்யவோ விரும்பினால் இந்த எண்ணை வைத்திருங்கள்.

        அன்புடன்,
        பசுமை கிராண்டே விடுதி
        '''

In [ ]:
departments = {
    'marketing': ['marketing', 'seo', 'community manager'],
    'sales': ['reservations', 'sales', 'booking'],
    'accountancy': ['accountancy', 'finance', 'purchase']
}

In [ ]:
def contact_dept(user_sent, departments):
  '''
  Takes the sentence and all the departments as input
  and returns the department email that the user wants to be contact with.
  '''
  email = None
  for k,v in departments.items():
    for d in user_sent:
      if d in v:
        email = f'{k}@greengrandeinn.com'
  return email

In [ ]:
def check_dates(arr, dep):
    arr_date = datetime.strptime(arr, "%d/%m/%Y").date()
    dep_date = datetime.strptime(dep, "%d/%m/%Y").date()
    if arr_date >= date.today():
        if dep_date > arr_date:
            return True
    return False

In [ ]:
model_dic = model.state_dict()
model = NeuralNet(input_size, hidden_size, output_size)
model.load_state_dict(model_dic)
model.eval()
current_datetime = datetime.now()
current_date_time = current_datetime.strftime("%m/%d/%Y, %H:%M:%S")
print(current_date_time)
bot_name = 'Green Grande Inn'
print("-------------------------------------------------------------------------GREEN GRANDE INN BOT-------------------------------------------------------------------------------")
print("Let's chat: type 'quit' to exit")

while True:
  sentence = input('You: ')
  if sentence == 'quit':
    break

  sentence = tokenize(sentence)
  X = bag_of_words(sentence, all_words)
  X = X.reshape(1, X.shape[0])
  X = torch.from_numpy(X)

  output = model(X)
  _,predicted = torch.max(output, dim=1)
  tag = tags[predicted.item()]

  probs = torch.softmax(output, dim=1)
  prob = probs[0][predicted.item()]
  todays = date.today()
  if prob.item() > 0.75:
    for intent in intents['intents']:
      if tag == intent['tag']:

        if tag == 'booking':
            print(f"{bot_name}: {random.choice(intent['responses'])}")
            # Stage 1: Customer's Name
            f_name = input('\tFirst Name: ')
            l_name = input('\tLast Name: ')

            # Stage 2: Booking Dates
            print('Please enter appropriate dates')
            arr = input('\tArrival day (DD/MM/YYYY): ')
            dep = input('\tDeparture day (DD/MM/YYYY): ')

            if check_dates(arr, dep):
                # Stage 3: Room and service
                room = input('\tWhich type of room are you looking for?: ')
                if room == 'deluxe':
                  rate = 2000
                else:
                  rate = 1000
                num_room = input('\thow many rooms do you want to book?: ')
                service = input('\tWhich service do you prefer?: ')
                # Stage 4: Confirmation and Final Rate
                c1 = Customer((f_name, l_name), (arr, dep), room, service, rate)
                c1.ID()
                all_bookings = all_bookings.append(c1.__dict__, ignore_index=True)
                print(c1)
            else:
                print('Invalid date selection. Please select appropriate dates.')
        elif tag == 'booking_t':
            print(f"{bot_name}: {random.choice(intent['responses'])}")
            # Stage 1: Customer's Name
            f_name = input('\tமுதல் பெயர்: ')
            l_name = input('\tகடைசி பெயர்: ')

            # Stage 2: Booking Dates
            print('பொருத்தமான தேதிகளை உள்ளிடவும்')
            arr = input('\tவருகை நாள் (DD/MM/YYYY): ')
            dep = input('\tபுறப்படும் நாள் (DD/MM/YYYY): ')

            if check_dates(arr, dep):
                # Stage 3: Room and service
                room = input('\tஎந்த வகையான அறையை நீங்கள் தேடுகிறீர்கள்?: ')
                if room == 'டீலக்ஸ்':
                  rate = 2000
                else:
                  rate = 1000
                num_room = input('\tஎத்தனை அறைகளை முன்பதிவு செய்ய விரும்புகிறீர்கள்? ')
                service = input('\tநீங்கள் எந்த சேவையை விரும்புகிறீர்கள்?: ')
                # Stage 4: Confirmation and Final Rate
                c1 = tamil_Customer((f_name, l_name), (arr, dep), room, service, rate)
                c1.ID()
                all_bookings = all_bookings.append(c1.__dict__, ignore_index=True)
                print(c1)
            else:
                print('தவறான தேதி தேர்வு. பொருத்தமான தேதிகளைத் தேர்ந்தெடுக்கவும்.')
        elif tag == 'cancellation':
            print(f"{bot_name}: {random.choice(intent['responses'])}")
            ref_num = input('\tReference number: ')
            if ref_num in all_bookings['id'].values:
              all_bookings = all_bookings.drop(all_bookings['id'][all_bookings['id'] == ref_num].index)
              print('Your reservation has been canceled.')
            else:
              print('This reference number does not exist.')
        elif tag == 'cancellation_t':
            print(f"{bot_name}: {random.choice(intent['responses'])}")
            ref_num = input('\tகுறிப்பு எண்: ')
            if ref_num in all_bookings['id'].values:
              all_bookings = all_bookings.drop(all_bookings['id'][all_bookings['id'] == ref_num].index)
              print('உங்கள் முன்பதிவு ரத்துசெய்யப்பட்டது.')
            else:
              print('இந்தக் குறிப்பு எண் இல்லை.')

        elif tag == 'contacts':
          contact_email = contact_dept(sentence, departments)
          if contact_email != None:
            print(f"{bot_name}: {random.choice(intent['responses'])} {contact_email}")
          else:
            print('Unfortunately this department does not exist.')
        elif tag == 'date':
          print(f"{bot_name}: {random.choice(intent['responses']).format(current_date_time=current_date_time)}")

        else:
          print(f"{bot_name}: {random.choice(intent['responses'])}")
        elif tag == 'contacts_t':
          contact_email = contact_dept(sentence, departments)
          if contact_email != None:
            print(f"{bot_name}: {random.choice(intent['responses'])} {contact_email}")
          else:
            print('Unfortunately this department does not exist.')
        elif tag == 'date_t':
          print(f"{bot_name}: {random.choice(intent['responses']).format(current_date_time=current_date_time)}")

        else:
          print(f"{bot_name}: {random.choice(intent['responses'])}")

  else:
    print(f'{bot_name}: I do not understand')

04/05/2023, 02:49:32
-------------------------------------------------------------------------GREEN GRANDE INN BOT-------------------------------------------------------------------------------
Let's chat: type 'quit' to exit
You: vanakam
Green Grande Inn: I do not understand
You: vannakam
Green Grande Inn: வணக்கம், நான் உங்களுக்கு எப்படி உதவ முடியும்
You: eppadi irukinga?
Green Grande Inn: வருகைக்கு நன்றி,நான் உங்களுக்கு எப்படி உதவ முடியும்
You: yaaravathu irukingala?
Green Grande Inn: வணக்கம்


KeyboardInterrupt: ignored